In [1]:
import functools
import time

from IPython.display import HTML, Image 
import gym

try:
  import brax
except ImportError:
  from IPython.display import clear_output 
  !pip install git+https://github.com/google/brax.git@main
  clear_output()
  import brax

from brax.v1 import envs
from brax.v1 import jumpy as jp
from brax.v1.envs import to_torch
from brax.v1.io import html
from brax.v1.io import image
import jax
from jax import numpy as jnp
import torch
v = torch.ones(1, device='cuda')  # init torch cuda before jax


/home/pcy/miniconda3/envs/jax/lib/python3.10/site-packages/flax/core/frozen_dict.py:169: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(


In [2]:

environment = "mydrone"  # @param ['ant', 'halfcheetah', 'hopper', 'humanoid', 'reacher', 'walker2d', 'fetch', 'grasp', 'ur5e']
env = envs.create(env_name=environment)
state = env.reset(rng=jp.random_prngkey(seed=0))

In [3]:

HTML(html.render(env.sys, [state.qp]))


In [4]:

%%time
rollout = []
for i in range(100):
  # wiggle sinusoidally with a phase shift per actuator
  # To change: action
  action = jp.sin(i * jp.pi / 15 + jp.arange(0, env.action_size) * jp.pi)
  state = env.step(state, action)
  rollout.append(state)
# jit compile env.step:
jit_env_step = jax.jit(env.step)
state = jit_env_step(state, jp.ones((env.action_size,)))


No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


CPU times: user 6.58 s, sys: 38.6 ms, total: 6.62 s
Wall time: 6.57 s


In [5]:

%%time
for _ in range(100):
  state = jit_env_step(state, jp.ones((env.action_size,)))


CPU times: user 1.42 s, sys: 504 µs, total: 1.42 s
Wall time: 1.38 s


In [6]:
HTML(html.render(env.sys, [s.qp for s in rollout]))